#### Plot nearest norms

This pack of functions serves as a tool to plot the nearest integer linear combinations of a given lcCube and its norms in a comprehensive figure. 

In [63]:
import itertools
import matplotlib.pyplot as plt
%run descent_tools.ipynb



def remove_coordinate_single(index, point):
    '''
    Remove a coordinate from a single point at the specified index.
    INPUTS:
        index (int): The index of the coordinate to be removed.
        point (list): The point represented as a list of coordinates.
    '''
    point_removed = point[:]
    point_removed.pop(index)
    return point_removed




def remove_coordinate(index, points, lcCube, lcLLL):
    '''
    Remove a coordinate from a list of points, lcCube, and lcLLL at the specified index.
    '''
    filtered_points = [remove_coordinate_single(index, point) for point in points]
    lcCubeRemoved = remove_coordinate_single(index, lcCube)
    lcLLLRemoved = remove_coordinate_single(index, lcLLL)
    return filtered_points, lcCubeRemoved, lcLLLRemoved




def get_index_to_remove(lcCube, lcLLL):
    '''
    Find the index of the identical coordinate shared by lcCube and lcLLL.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
    '''
    if len(lcCube) != len(lcLLL):
        raise ValueError("Both input lists must have the same length")

    for index, (elem_lcCube, elem_lcLLL) in enumerate(zip(lcCube, lcLLL)):
        if elem_lcCube == elem_lcLLL:
            return index
    return ValueError("The points do not share identical coordinate.")




def generate_nearby_integer_points(lcCube, exception_index, dist):
    '''
    Generate nearby integer points of lcCube within a given distance, excluding a specific index.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        exception_index (int): The index of the coordinate to exclude from the calculation.
        dist (int): The maximum distance to generate nearby integer points.
    '''
    n = len(lcCube)
    nearby_points = []
    
    # Calculate the range of possible integer coordinates for each dimension, except at index ind
    ranges = [range(int(round(coord) - dist + 1), int(round(coord) + dist)) if i != exception_index else [lcCube[exception_index]] for i, coord in enumerate(lcCube)]

    # Generate all possible combinations of integer coordinates within the ranges
    coordinate_combinations = list(itertools.product(*ranges))

    for point in coordinate_combinations:
        total_diff = sum(abs(lcCube[i] - point[i]) for i in range(n))
        if total_diff < dist:
            nearby_points.append(list(point))
    
    return nearby_points




def plot_nearest_norms(lcCube, lcLLL, B, neighborhood):
    '''
    Plot the nearest norms of lcCube and lcLLL, as well as nearby integer points.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
        B (list of lists): The basis matrix.
        neighborhood (int): The maximum distance to generate nearby integer points.
    '''
    index = get_index_to_remove(lcCube, lcLLL)
    nearby_integer_lcs = generate_nearby_integer_points(lcCube, index, neighborhood)

    nearby_removed, lcCubeRemoved , lcLLLRemoved= remove_coordinate(index, nearby_integer_lcs, lcCube, lcLLL)

    x_coords = [point[0] for point in nearby_removed]
    y_coords= [point[1] for point in nearby_removed]
    norms = [(vector(point)*matrix(B)).norm().n(digits=3) for point in nearby_integer_lcs]

    norm_min = min(norms)
    norm_max = max(norms)
    colors = [(norm - norm_min) / (norm_max - norm_min) for norm in norms]
    plt.scatter(x_coords, y_coords, c=colors, cmap='Blues', edgecolors='face', s=30)

    # plt.scatter(x_coords, y_coords, color='blue',s=30)
    plt.scatter(lcCubeRemoved[0], lcCubeRemoved[1], color="red", s=40)
    plt.scatter(lcLLLRemoved[0], lcLLLRemoved[1], color="green", s=40)

    # Add norm as a description to each point with coords [x, y]
    for x, y, norm in zip(x_coords, y_coords, norms):
        plt.annotate(
            f"{norm}",
            xy=(x, y),
            xytext=(x + 0.1, y + 0.1),
            fontsize=10,
        )

    # Annotate the lcCube's norm
    plt.annotate(
            vector(lcCube).norm().n(digits=3),
            xy=(lcCubeRemoved[0], lcCubeRemoved[1]),
            xytext=(lcCubeRemoved[0] + 0.1, lcCubeRemoved[1] + 0.1),
            fontsize=10,
            color="red",
        )    
    
    # Add gridlines corresponding to integers
    plt.grid(True, which='both')

    # Set axis ticks to integers only
    plt.xticks(range(int(min(x_coords)) - 1, int(max(x_coords)) + 2))
    plt.yticks(range(int(min(y_coords)) - 1, int(max(y_coords)) + 2))
    
    tmp = B[:]
    tmp.pop(index)

    # Show the plot
    print("Deleted index: ", index)
    print("x-axis: vector {} with norm {}".format(tmp[0], vector(tmp[0]).norm().n(digits=4)))
    print("y-axis: vector {} with norm {}".format(tmp[1], vector(tmp[1]).norm().n(digits=4)))
    plt.show()

#### Plot norm function

Plots the function $f(x) = x \cdot G\cdot x^T$ in 3D, where $x$ is a vector $(x_1, ... ,1, x_{i+1}, ... x_n)$ where the $i$th varible is ommited and replaced with 1.



In [9]:
from sage.all import *
from IPython.display import display, Math

# Define the matrix G

def plot_norm_function(G, lcCube, lcLLL, rng=5):
    '''
    Plot the nearest norms of lcCube and lcLLL, as well as nearby integer points.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
        B (list of lists): The basis matrix.
        neighborhood (int): The maximum distance to generate nearby integer points.
    '''
    G = matrix(G)
    n = G.nrows()

    index = get_index_to_remove(lcCube, lcLLL)
    variables = [var('x_%d' % (j+1)) for j in range(n) if j != index]

    u = vector([var('x_%d' % (j+1)) if j != index else 1 for j in range(n)])
    f = u * G * u # transpose here is not neeeded, sage deals with this syntax


    print("lccube:", lcCube)
    print("lcLLL:", lcLLL)
    print("x: ",u)
    print("Deleted index: ", index, " (python indexing)")
    print(G)
    display(Math(latex(f.simplify_full()))) # printing the function f

    p = plot3d(f, (variables[0], -rng, rng), (variables[1], -rng, rng))
    p.show()


: 

#### matrix_multiplication_detailed

`matrix_multiplication_detailed`
Given a matrix $A$ and a vector $v$, computes a detailed matrix multiplication $v \cdot A \cdot v^T$ in a sense, that every cell in the matrix is multiplicated with the corresponding cell of the vector. The goal is to have clearer perspective on which cells in the matrix are effecting the multiplication the most. 
Return this detailed matrix and a sum of it's elements, resp. the result of $v \cdot A \cdot v^T$.

In [ ]:

def matrix_multiplication_detailed(A, vector) -> (matrix, float):
    """
    OUTPUT
        A - matrix A multiplied in rows and cols by vector
        x - sum of all elements in the returned matrix
    """
    A = matrix_to_list(A)
    n = len(A)
    vector = vector_to_list(vector)
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j]*  vector[i] * vector[j], 2)
    return matrix(A), round(sum(sum(A, [])), 2)